<a href="https://colab.research.google.com/github/DavidCastroPena/Vaswani2017/blob/main/replicatingAttentionIsAllWhatYouNeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


# when we are using a transformer to analyze say a sentence ("the cat sat"), first what happens is that each word gets stored in a
# query: q_sat. Now each query or what i am looking to analize is compared (dot product) with its all the keys; all words have keys: k_the, k_cat,
# and k_sat. The dot product geometrically shows that if the direction of two vectors is related, this will reflect a large dot
#product, which shows that the two tokens are related

"""
qsat⋅kthe= 0.1
	​
𝑞sat⋅𝑘cat=2.1

qsat⋅kcat=1.5

"""

# "sat" is more related to cat than the

#Recall, the keys and query are linear transformation of their embeddings. Intuitively, we say that the query is a question
#per token that aims to uncover the role of the specific word in a given text. The word “question” is shorthand for
#something very precise: The query defines a direction in vector space along which relevance is measured.



#Lesson 1 goal: implement and understand:

#Attention(Q,K,V)=softmax(QK^T/root(d_k)*V

#Key concepts:

#PyTorch tensors

#Matrix multiplication

#Softmax

#Masking

#Shape reasoning


#Setup
import torch
import torch.nn.functional as F

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


device: cpu


In [4]:
import torch

# Define dimensions for the SelfAttentionHead
d_model = 64  # Embedding dimension
d_k = 32      # Dimension of Query and Key vectors
d_v = 32      # Dimension of Value vectors

# Instantiate the SelfAttentionHead
attention_head = SelfAttentionHead(d_model, d_k, d_v).to(device)

# Create a dummy input tensor (batch_size, sequence_length, d_model)
batch_size = 2
sequence_length = 5
X = torch.randn(batch_size, sequence_length, d_model, device=device)

# Perform a forward pass
out, attn = attention_head(X)

print("Input X shape:", X.shape)
print("Output 'out' shape:", out.shape)
print("Attention 'attn' shape:", attn.shape)

# Verify row sums of attention matrix (should be ~1)
print("\nAttention matrix row sums (first batch, first head, should be ~1):\n", attn[0].sum(dim=-1).detach().cpu())

Input X shape: torch.Size([2, 5, 64])
Output 'out' shape: torch.Size([2, 5, 32])
Attention 'attn' shape: torch.Size([2, 5, 5])

Attention matrix row sums (first batch, first head, should be ~1):
 tensor([1., 1., 1., 1., 1.])


In [2]:

# Q,K ,V, and a mask are tensor. A tensor is a multi-dimensional array of numbers and in transformers all the meaning
#arises from learned tensor transformations and interactions

# embeddings are vectors, but to make computations or to represent sentences, instead of converting to
# vectors in a sequential way or one by one, tensors allow you to represent text in a more efficient way


# A vector is a tensor (a 1-D tensor). A matrix is a tensor (a 2-D tensor). So the real question is:

# Why do we need higher-dimensional tensors instead of just one vector at a time? Imagine you process one word at a time,
# using vectors only. Sentence: “The cat sat”.You would have to do this:

# Take embedding of "The" → vector compare it to "cat" → vector

# Compare it to "sat" → vector

# Repeat for "cat"

# Repeat for "sat"

# That’s nested loopsnsequential computation very slow hard to parallelize messy gradients. This is basically how early RNNs worked.

# What tensors give you: structure Tensors let you represent many things at once. Instead of:

# “One word → one vector” You represent: All words, all positions, all heads, all batches — at the same time


def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Q: (batch, heads, q_len, d_k)
    K: (batch, heads, k_len, d_k)
    V: (batch, heads, k_len, d_v)
    mask: (batch, heads, q_len, k_len) with 1 for allowed, 0 for blocked (optional)

    returns:
      out: (batch, heads, q_len, d_v)
      attn: (batch, heads, q_len, k_len)
    """
    d_k = Q.size(-1)

    # (batch, heads, q_len, k_len)

  #When you compute qverb*kj, you are asking:
  # "does token k, in its current representation, lie in a directaion that is useful for a verb right now?"
  # if the answer is yes (large dot product), attention weight increases, information form that token flows


    scores = Q @ K.transpose(-2, -1) / (d_k ** 0.5)

    if mask is not None:
        # Set blocked positions to a large negative value so softmax ~ 0 there.
        scores = scores.masked_fill(mask == 0, -1e9)

#the softmax ensure converting the raw scores into probabilities of how related or important are
#the tokens to each other. However, it does not have the information itself: recall that q
#contains the question the word asks about its role, the K decide how to be matched, and V decide what
# information to contribute.

# Consider the word "bank". For matching, you may want to match syntactic role or entity type.
# For content, you may want to pass financial or river meaning. In this case, Kj encodes:
# “If another token is looking for this kind of information, how strongly should I respond?” This response
#is continuous, context-dependet, and learned statistically

# Sentence “The cat sat.” During training, the model repeatedly observes that: verbs benefit from attending to certain tokens
# those tokens often share embedding patterns (things we humans call “nouns”)
# Over time:

  # Wk learns to map those tokens into a region of space
  # that region is useful when queried by verbs

#  learns to map those tokens into a region of space,that region is useful when queried by verbs

# But the model never learns:

# “This is a noun”

# It learns: “Tokens with embeddings like this tend to be useful when verbs query the context.” That’s a relational regularity, not a category.


    attn = F.softmax(scores, dim=-1)

# # the multiplication between the attn and v: The multiplication of attention weights by values computes a learned,
# conditional linear combination of representations — mathematically similar to a regression or mixture model —
# but embedded inside a deep, nonlinear system.


    out = attn @ V
    return out, attn



In [5]:
# Lets create a small example to see the attention at work

batch, heads, seq_len, d_k, d_v = 1, 1, 4, 3, 2

Q = torch.randn(batch, heads, seq_len, d_k, device=device)
K = torch.randn(batch, heads, seq_len, d_k, device=device)
V = torch.randn(batch, heads, seq_len, d_v, device=device)

out, attn = scaled_dot_product_attention(Q, K, V)

print("Q shape:", Q.shape)
print("attn shape:", attn.shape)
print("out shape:", out.shape)

print("\nAttention matrix (q_len x k_len):\n", attn[0,0].detach().cpu())
print("\nRow sums (should be ~1):\n", attn[0,0].sum(dim=-1).detach().cpu())


Q shape: torch.Size([1, 1, 4, 3])
attn shape: torch.Size([1, 1, 4, 4])
out shape: torch.Size([1, 1, 4, 2])

Attention matrix (q_len x k_len):
 tensor([[0.3625, 0.2162, 0.2355, 0.1857],
        [0.0161, 0.6331, 0.3333, 0.0176],
        [0.0778, 0.0909, 0.1192, 0.7121],
        [0.7052, 0.0087, 0.0229, 0.2631]])

Row sums (should be ~1):
 tensor([1.0000, 1.0000, 1.0000, 1.0000])


# 📌 NOTE — Bridging the gap: from scaled dot-product attention to a Self-Attention Head

At this point in the replication, it is **normal to feel lost**.

Why?  
Because we moved from:
- understanding **attention as a mathematical operation**
to:
- seeing **attention embedded inside a neural network module**

This note explains **exactly how those two relate**, step by step.

---

## 1️⃣ What we implemented first: scaled dot-product attention (pure math)

We started by implementing:

Attention(Q, K, V) = softmax(QKᵀ / √dₖ) · V

This function:

- takes **Q, K, V as inputs**
- computes similarity scores via dot products
- normalizes them with softmax
- uses them to compute a weighted sum of values

At this stage:
- Q, K, V were **dummy tensors** (random numbers)
- the goal was to understand:
  - tensor shapes
  - matrix multiplication
  - softmax
  - attention matrices
  - how information flows

This step was **intentional** and **necessary**.

👉 Key point:
> `scaled_dot_product_attention` is a **standalone mathematical operator**.
> It does NOT know where Q, K, V come from.

Think of it as the **engine** of attention.

---

## 2️⃣ The missing question: where do Q, K, V come from in a real Transformer?

In the actual Transformer paper:

- Q, K, V are **not given directly**
- They are **computed from token embeddings**

The real pipeline is:

tokens  
→ embeddings X  
→ linear projections  
→ Q, K, V  
→ attention

Mathematically:

Q = X · W_Q  
K = X · W_K  
V = X · W_V  

Where:
- X is the embedding tensor (B × T × d_model)
- W_Q, W_K, W_V are **learned weight matrices**

So Q, K, V are:
- different *views* of the same tokens
- learned automatically during training

---

## 3️⃣ Why `SelfAttentionHead` exists

The purpose of `SelfAttentionHead` is **not to introduce new math**.

Its only job is to:

1. Take token embeddings `X`
2. Create Q, K, V using learned linear layers
3. Call `scaled_dot_product_attention(Q, K, V)`
4. Return:
   - contextualized token representations
   - the attention matrix

Conceptually:

SelfAttentionHead(X)
    ├── computes Q = XW_Q
    ├── computes K = XW_K
    ├── computes V = XW_V
    └── calls scaled_dot_product_attention(Q, K, V)

So:

- `scaled_dot_product_attention` = the **math**
- `SelfAttentionHead` = the **first real Transformer component**

Nothing magical is happening here.

---

## 4️⃣ Why new tensor dimensions suddenly appear (the “heads” dimension)

The attention function is written to support **multi-head attention** later.

For that reason, it expects tensors shaped like:

(Batch, Heads, SequenceLength, Dimension)

Even when:
- batch = 1
- heads = 1

So inside `SelfAttentionHead`, we do:

Q = Q.unsqueeze(1)

This:
- does NOT change the meaning of Q
- only adds a placeholder dimension for heads
- allows the same function to work for 1 head or many heads

👉 This is a **software design choice**, not a conceptual change.

---

## 5️⃣ Why this step is part of the replication plan

This step corresponds to **Section 3.2.1** of the paper:
> “Scaled Dot-Product Attention”

But now implemented as:
- a **learnable module**
- not just a formula on paper

At this point in the replication, we are still in:

✅ Step 1 — Build attention correctly

We are NOT yet:
- stacking layers
- using multiple heads
- training a model
- optimizing performance

We are simply answering:

> “How does a Transformer turn embeddings into attention outputs?”

---

## 6️⃣ Key takeaway (lock this in)

Nothing new was introduced conceptually.

We only:
- connected the attention math
- to learnable parameters
- in a reusable module

If you understand:
- how scaled dot-product attention works
- and that `SelfAttentionHead` just *creates Q, K, V*

then you understand this part of the Transformer.

---

## 7️⃣ What comes next (but not yet)

Only AFTER this is fully clear do we move on to:
- masking (causal attention)
- multi-head attention
- stacking layers

We are still building the foundation.


In [11]:
# Explaining the  make causal mask: attn[i, j] = how much token i (the query) attends to token j (the key/value).

# Because you used a causal mask, token i is only allowed to attend to tokens ≤ i (no looking ahead).

# That’s why everything above the diagonal is zero.

def make_causal_mask(B, H, T, device):
    # lower-triangular matrix: 1 = allowed, 0 = blocked
    m = torch.tril(torch.ones(T, T, device=device))
    # expand to (B, H, T, T)
    return m.view(1, 1, T, T).expand(B, H, T, T)


In [13]:


# Testing the causal masking: torch.manual_seed(0)
# Interpreting results:
# [0.5445, 0.4555, 0.0000, 0.0000]
# ~54% to token 0

# Exactly expected.

device = "cuda" if torch.cuda.is_available() else "cpu"

B, T, d_model, d_k, d_v = 1, 4, 8, 4, 4
X = torch.randn(B, T, d_model, device=device)

head = SelfAttentionHead(d_model, d_k, d_v).to(device)

mask = make_causal_mask(B, 1, T, device)  # (B, H=1, T, T)

out, attn = head(X, mask=mask)

print("attn:\n", attn[0].detach().cpu())
print("row sums:\n", attn[0].sum(dim=-1).detach().cpu())


attn:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5559, 0.4441, 0.0000, 0.0000],
        [0.2983, 0.2798, 0.4219, 0.0000],
        [0.1572, 0.2115, 0.3404, 0.2909]])
row sums:
 tensor([1.0000, 1.0000, 1.0000, 1.0000])


In [10]:


import torch
import torch.nn as nn
import torch.nn.functional as F

# This class implements a SINGLE self-attention head: he SelfAttentionHead class, which inherits from nn.Module,
# represents a concrete component within a neural network. Its primary responsibility is to create the Q, K, and V tensors from the input embeddings X using learnable linear projections (Wq, Wk, Wv).
# Why it's a class (inheriting from nn.Module): Classes inheriting from nn.Module in PyTorch are designed for components that:
# Contain Learnable Parameters: The Wq, Wk, and Wv matrices are nn.Linear layers, which are learnable. nn.Module manages these parameters, making them discoverable by optimizers during training.
# Define a forward pass: The forward method encapsulates the sequence of operations (creating Q/K/V, calling scaled_dot_product_attention, handling dimensions) that this specific module performs.
# Are part of a larger graph: By being an nn.Module, it can be easily integrated into a larger neural network (like a full Transformer block), allowing for proper dependency tracking and automatic differentiation.

class SelfAttentionHead(nn.Module):

    def __init__(self, d_model: int, d_k: int, d_v: int):
        super().__init__()

# At this point, we are passing from embeddings per each token to the matrices WQ,WK, and V learned matrices.
# The learned
        # Linear projection that maps embeddings -> query vectors
        # Wq is a learned matrix of shape (d_model, d_k)
        self.Wq = nn.Linear(d_model, d_k, bias=False)

        # Linear projection that maps embeddings -> key vectors
        # Wk is a learned matrix of shape (d_model, d_k)
        self.Wk = nn.Linear(d_model, d_k, bias=False)

        # Linear projection that maps embeddings -> value vectors
        # Wv is a learned matrix of shape (d_model, d_v)
        self.Wv = nn.Linear(d_model, d_v, bias=False)
#X: (B, T, d_model) which is a fundamental way we describe tensors in deep learning, especially when working with sequences
# like text. Let's break down each part:X: This is the input tensor itself. In the context of a Transformer,
# it usually represents a batch of token embeddings.

# B (Batch Size):

# This is the number of independent sequences (e.g., sentences, phrases, or documents) that are being processed together.
# Modern neural networks, especially when using GPUs, process data in 'batches' rather than one item at a time.
#This is much more efficient because it allows for parallel computations.
# Example: If B=3, it means you're feeding three different sentences into the model simultaneously.

#T (Sequence Length / Number of Tokens):

# This is the length of each sequence in the batch. It represents how many tokens (words, sub-words, or characters,
# depending on your tokenization) are in each individual input sequence.
# Example: If T=10, it means each of your B sentences has 10 tokens.
# d_model (Model Dimension / Embedding Dimension):

# This is the dimensionality of the vector used to represent each single token. Every token (like a word) is
#converted into a numerical vector of this size. This vector is meant to capture the semantic and positional
#information of that token. Example: If d_model=512, then each token in your T length sequences is represented
#by a vector of 512 numbers.

    def forward(self, X, mask=None):
        """
        X: (B, T, d_model)
        returns:
          out:  (B, T, d_v)
          attn: (B, T, T)
        """

        # Project embeddings -> Q/K/V
        Q = self.Wq(X)  # (B, T, d_k)
        K = self.Wk(X)  # (B, T, d_k)
        V = self.Wv(X)  # (B, T, d_v)

        # Add head dimension so we can reuse the same attention function
        Q = Q.unsqueeze(1)  # (B, 1, T, d_k)
        K = K.unsqueeze(1)  # (B, 1, T, d_k)
        V = V.unsqueeze(1)  # (B, 1, T, d_v)

        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)

        # Remove head dimension (since H=1 here)
        return out.squeeze(1), attn.squeeze(1)

# 📌 NOTE — The correct mental model: architecture vs learning in the Transformer

At this point, it is important to separate **what is being built** from **what is being learned**.

The Transformer paper combines mathematics, architecture, and training in one narrative.
In this notebook, we are intentionally **separating them**, which is why it may feel like
“nothing is being learned yet”.

This note clarifies the correct mental model.

---

## 1️⃣ Three distinct layers of understanding (do not mix them)

When reading *Attention Is All You Need*, there are **three different layers** happening at once:

---

### Layer 1 — The mathematical operator (pure attention math)

This is the formula:

Attention(Q, K, V) = softmax(QKᵀ / √dₖ) · V

At this layer:
- Q, K, V are just tensors
- no corpus is required
- no learning is required
- this is pure linear algebra

This is what we implemented first with:
- dummy Q, K, V
- random tensors
- shape checks
- attention matrices

👉 Purpose: understand **how attention works mechanically**

---

### Layer 2 — The architectural module (parameterized attention)

This is where `SelfAttentionHead` appears.

At this layer:
- we introduce **learnable matrices** W_Q, W_K, W_V
- Q, K, V are computed as:
  
  Q = X · W_Q  
  K = X · W_K  
  V = X · W_V  

- X are token embeddings
- W_Q, W_K, W_V are **trainable parameters**

Important:
- these matrices are **randomly initialized**
- they are *capable* of learning
- but they have **not learned anything yet**

👉 Purpose: build the **Transformer architecture**, not train it

This corresponds to the *architecture description* in the paper (Section 3).

---

### Layer 3 — Training (where learning actually happens)

Learning only happens when we introduce:

- a dataset (toy task or real text corpus)
- a prediction objective (e.g., next-token prediction)
- a loss function
- backpropagation
- an optimizer

Only then do:
- W_Q, W_K, W_V
- embeddings
- all other parameters

begin to change in response to data.

👉 Purpose: make attention **meaningful**

This corresponds to the *training procedure* sections of the paper.

---

## 2️⃣ What we are doing right now in this notebook

We are currently **between Layer 1 and Layer 2**.

Specifically:
- we fully understand the attention math (Layer 1)
- we have built the parameterized attention module (Layer 2)
- we have NOT introduced data, loss, or optimization (Layer 3)

So it is **expected and correct** that:
- W_Q, W_K, W_V exist
- but they do not encode syntax, semantics, or grammar
- attention patterns look random

This is not a bug.
This is the correct state at this stage.

---

## 3️⃣ Why we are not using a corpus yet

We intentionally do NOT start with real text because:

- bugs in attention math are hard to debug once training is added
- shape errors become opaque
- attention feels like “magic” instead of computation

The correct replication order is:

1. Build attention
2. Verify attention
3. Build the Transformer block
4. Train on **toy tasks**
5. Only then train on real text

This is exactly the plan we are following.

---

## 4️⃣ How the paper’s wording can be misleading

When the paper says:
> “We use learned linear projections to obtain the queries, keys, and values…”

They are describing the **final trained system**, not the initial state of the code.

In code:
- those projections exist immediately
- but they only become “learned” after training

---

## 5️⃣ Key takeaway (lock this in)

> **Right now, we are building the machine.  
> Learning only begins once we add a dataset, a loss, and optimization.**

If you understand:
- how attention works mathematically
- how Q/K/V are produced architecturally
- and why learning has not started yet

then you have the correct mental model.

---

## 6️⃣ What comes next

Before touching real text, we will:
- finish attention (masking)
- add multi-head attention
- stack layers
- then train on simple toy tasks

Only after that will we introduce a real corpus.

This is the correct and disciplined way to replicate the paper.


In [14]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention (MHA)

    Input:
      X: (B, T, d_model)

    Output:
      out:  (B, T, d_model)
      attn: (B, h, T, T)   # attention matrix per head
    """
    def __init__(self, d_model: int, num_heads: int):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.h = num_heads
        self.d_k = d_model // num_heads  # dimension per head (queries/keys/values per head)

        # Big projections: X -> Q,K,V all in d_model space
        # Each contains all heads packed together.
        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)

        # Final output projection: concat(heads) -> d_model
        self.Wo = nn.Linear(d_model, d_model, bias=False)

    def forward(self, X, mask=None):
        """
        X: (B, T, d_model)

        mask: should be broadcastable to (B, h, T, T)
              - for causal masking you can build (B, 1, T, T) and it will broadcast across heads
        """
        B, T, _ = X.shape

        # 1) Project X into packed Q, K, V
        # Shapes: (B, T, d_model)
        Q = self.Wq(X)
        K = self.Wk(X)
        V = self.Wv(X)

        # 2) Reshape packed (B, T, d_model) -> (B, h, T, d_k)
        # - view splits the last dimension into (h, d_k)
        # - transpose makes heads dimension come before time
        Q = Q.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)
        K = K.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)
        V = V.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)

        # 3) Apply scaled dot-product attention per head (in parallel)
        # out:  (B, h, T, d_k)
        # attn: (B, h, T, T)
        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)

        # 4) Concatenate heads: (B, h, T, d_k) -> (B, T, d_model)
        out = out.transpose(1, 2).contiguous().view(B, T, self.d_model)

        # 5) Final linear projection back to d_model
        out = self.Wo(out)

        return out, attn


In [15]:
#Testing multihead

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

B, T, d_model, h = 1, 4, 8, 2
X = torch.randn(B, T, d_model, device=device)

mha = MultiHeadAttention(d_model=d_model, num_heads=h).to(device)

# causal mask: (B, 1, T, T) -> broadcasts to (B, h, T, T)
mask = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T).expand(B, 1, T, T)

out, attn = mha(X, mask=mask)

print("X:", X.shape)        # (1, 4, 8)
print("out:", out.shape)    # (1, 4, 8)
print("attn:", attn.shape)  # (1, 2, 4, 4)

print("\nHead 0 attention:\n", attn[0,0].detach().cpu())
print("\nHead 0 row sums:\n", attn[0,0].sum(dim=-1).detach().cpu())

print("\nHead 1 attention:\n", attn[0,1].detach().cpu())
print("\nHead 1 row sums:\n", attn[0,1].sum(dim=-1).detach().cpu())


X: torch.Size([1, 4, 8])
out: torch.Size([1, 4, 8])
attn: torch.Size([1, 2, 4, 4])

Head 0 attention:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5478, 0.4522, 0.0000, 0.0000],
        [0.3274, 0.2866, 0.3861, 0.0000],
        [0.2485, 0.2721, 0.2112, 0.2682]])

Head 0 row sums:
 tensor([1., 1., 1., 1.])

Head 1 attention:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4438, 0.5562, 0.0000, 0.0000],
        [0.3240, 0.3522, 0.3239, 0.0000],
        [0.2426, 0.2616, 0.2515, 0.2442]])

Head 1 row sums:
 tensor([1.0000, 1.0000, 1.0000, 1.0000])
